In [5]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
# Set parameters for image dimensions, batch size, and number of classes
batch_size = 32
image_height = 128
image_width = 128
num_classes = 3
num_epochs = 20
dropout_rate = 0.5
l2_reg = 0.01

In [9]:
# Build the CNN model
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_reg), input_shape=(image_height, image_width, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(256, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(l2_reg)),
    layers.Dropout(dropout_rate),
    layers.Dense(num_classes, activation='softmax')
])

In [11]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [13]:
# Load and preprocess the image data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

In [15]:
train_data_dir = "D:/IMAGES_DATASET/Train"

# Set up the training data generator
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

Found 15 images belonging to 3 classes.


In [17]:
# Set up the validation data generator
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 3 images belonging to 3 classes.


In [21]:
# Define callbacks for better training
checkpoint = callbacks.ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

In [23]:
# Train the model
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=num_epochs,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

C:\Users\Adnan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.4000 - loss: 9.3038

C:\Users\Adnan\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.4000 - loss: 9.3038 - val_accuracy: 0.3333 - val_loss: 7.5199 - learning_rate: 0.0010
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2000 - loss: 9.9273 - val_accuracy: 0.6667 - val_loss: 7.5095 - learning_rate: 0.0010
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 709ms/step - accuracy: 0.4000 - loss: 8.9104 - val_accuracy: 0.3333 - val_loss: 7.5308 - learning_rate: 0.0010
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 625ms/step - accuracy: 0.4000 - loss: 10.4568 - val_accuracy: 0.3333 - val_loss: 7.5295 - learning_rate: 0.0010
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 813ms/step - accuracy: 0.3333 - loss: 8.6032 - val_accuracy: 0.3333 - val_loss: 7.5215 - learning_rate: 0.0010
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 688ms/step - accuracy: 0.4667 - loss: 9.6988 - val_accuracy: 0.3333 - val_loss: 7.5243 - learning_rate: 2.0000e-04
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 837ms/step - accuracy: 0.3333 - loss: 9.2213 - val_accuracy: 0.3333 - v

In [24]:
# Evaluate the model on the test data
test_data_dir = 'path/to/test/directory'
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(image_height, image_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy}")

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'path/to/test/directory'

In [26]:
# Predict labels for new images
new_image_path = 'path/to/new/image.jpg'
new_image = tf.keras.preprocessing.image.load_img(new_image_path, target_size=(image_height, image_width))
new_image = tf.keras.preprocessing.image.img_to_array(new_image)
new_image = new_image / 255.0  # Normalize the image
new_image = tf.expand_dims(new_image, axis=0)  # Add batch dimension

FileNotFoundError: [Errno 2] No such file or directory: 'path/to/new/image.jpg'